<a href="https://colab.research.google.com/github/Nakib-Nasrullah/Heart_disease/blob/main/ecg_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# Imoort libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import(
    accuracy_score, confusion_matrix, classification_report,
    precision_score, recall_score, f1_score,
    roc_curve, roc_auc_score, auc
)

In [30]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/ecg_dataset_process.csv')

In [32]:
# Basic exploration
print("Dataset shape:", df.shape)
print("n\First 5 rows:")
print(df.head())

print("\nDataset Info:")
print(df.info())


Dataset shape: (7610, 13)
n\First 5 rows:
   patientAge  P_wave  Q_wave  R_wave  S_wave  T_wave  PR_Interval  \
0          22      80      58     146      51      73         0.22   
1          20      78      57     142      50      71         0.21   
2          22      78      57     142      50      71         0.21   
3          23      78      57     142      50      71         0.21   
4          24      78      57     142      50      71         0.21   

   QRS_Complex  RR_Interval  QT_Interval  QTc_Interval      Disease  \
0         0.07          0.5         0.40          0.57  Tachycardia   
1         0.07          1.0         0.38          0.39       Normal   
2         0.07          0.9         0.38          0.39       Normal   
3         0.07          0.9         0.38          0.39       Normal   
4         0.07          0.9         0.38          0.39       Normal   

   Risk_level  
0           1  
1           0  
2           0  
3           0  
4           0  

Dataset Info:

In [33]:
print("\nBaasic statistics:")
print(df.describe())

print("\nCheck for missing values:")
print(df.isnull().sum())


Baasic statistics:
        patientAge        P_wave        Q_wave        R_wave        S_wave  \
count  7610.000000   7610.000000   7610.000000   7610.000000   7610.000000   
mean     22.556110     66.677792     48.221551    121.654928     42.155716   
std       2.995299    505.526512    367.703410    919.135621    321.728603   
min      18.000000      1.000000    -43.000000      2.000000    -25.000000   
25%      22.000000     50.000000     36.000000     90.000000     31.000000   
50%      22.000000     57.000000     42.000000    104.000000     36.000000   
75%      23.000000     72.000000     52.000000    130.000000     46.000000   
max      47.000000  43728.000000  31802.000000  79506.000000  27827.000000   

             T_wave  PR_Interval  QRS_Complex  RR_Interval  QT_Interval  \
count   7610.000000  7610.000000  7610.000000  7610.000000  7610.000000   
mean      60.654796     0.184907     0.066530     1.084465     0.343012   
std      459.574684     1.378839     0.461826     9.

In [34]:
print("\nTarget variable distribution:")
print(df['Risk_level'].value_counts())


Target variable distribution:
Risk_level
0    6670
1     940
Name: count, dtype: int64


In [35]:
print("\nTarget variable distribution:")
df.columns = df.columns.str.strip()
print(df['Disease'].value_counts())


Target variable distribution:
Disease
Normal         6670
Tachycardia     911
Bradycardia      29
Name: count, dtype: int64


In [36]:
# train (70%) validation (10%) test(20%)

#Separate input (X) and output (y)
X = df.drop('Risk_level', axis=1) # Everything except Risk_level
y = df['Risk_level']              # Only Risk_level

print("X has all patients information (13 colums)")
print("y has Risk_level (1 column)")

# Split: 80% for training, 20% for testing

#Train test split function is very importent for machine learning

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.2,     # 20% for test
    random_state = 20   # Same result every time # random_state mens in every time same data go to train and same data go to test.
)

print(f"\n We will train with: {len(X_train)} patients")
print(f"We will test with: {len(X_test)} patients")

X has all patients information (13 colums)
y has Risk_level (1 column)

 We will train with: 6088 patients
We will test with: 1522 patients
